
# <div style="color: lightgreen; text-align: center;">  Funciones AWS Lambda </div>

### <div style="color: lightgreen; text-align: center;">  _____________________________ </div>

## <div style="color: lightgreen; text-align: center;"> .  def developer  . </div>

import awswrangler as wr
import boto3
import json

def lambda_handler(event, context):
    # Obtener el nombre de la desarrolladora del evento
    developer_name = event['dev']
    
    # Conectarse al bucket de S3 donde se encuentra el archivo Parquet
    s3 = boto3.client('s3')
    
    # Descargar el archivo Parquet desde S3
    df = wr.s3.read_parquet("s3://henry-lab-142024/dev/", dataset=True)
    
    df2 = df[df['developer'] == developer_name]
    
    years_list = sorted(df2['release_date'].dt.year.unique(), reverse=True)
    
    lista_final = []
    
    # Creo el bucle para llenar la lista con los diccionarios
    for year in years_list:
        df_year = df2[df2['release_date'].dt.year == year]
        c_items = int(df_year['item_id'].count())  # Contar los items
        free = int(df_year[df_year['price'] == 0]['item_id'].count())  # Contar los items con precio 0
        m = {str(year): [{'Cantidad de items': c_items}, {'Contenido Free': free}]}
        lista_final.append(m)
        
    result = json.dumps(lista_final)
    return result

## <div style="color: lightgreen; text-align: center;"> .  def best_developer_year  . </div>


import awswrangler as wr
import boto3
import json

def lambda_handler(event, context):
    # Obtener el nombre de la desarrolladora del evento
    fecha = int(event['year'])
    
    # Conectarse al bucket de S3 donde se encuentra el archivo Parquet
    s3 = boto3.client('s3')
    
    # Descargar el archivo Parquet desde S3
    df = wr.s3.read_parquet("s3://henry-lab-142024/dev/", dataset=True)
    
    # Filtro el db con por el año que voy a revisar.
    b_dev = df[df['release_date'].dt.year == fecha]
    
    # Agrupo por la columna developer y sumo los valores de la columna 'positivo'
    dev_reviews = b_dev.groupby('developer')['positivo'].sum()
    
    top_devs = list(dev_reviews.nlargest(3).keys())
   
   
    puesto1 = top_devs[0]
    puesto2 = top_devs[1]
    puesto3 = top_devs[2]
    
    top_list = [{"Puesto 1": puesto1}, {"Puesto 2": puesto2}, {"Puesto 3": puesto3}]
    
    # Convertir la lista a JSON
    result = json.dumps(top_list)
    
    return result


## <div style="color: lightgreen; text-align: center;"> .  def developer_reviews_analysis  . </div>

import json
import awswrangler as wr
import boto3



def lambda_handler(event, context):
    
    # Obtener el nombre de la desarrolladora del evento
    developer_name = event['dev']
    
    # Conectarse al bucket de S3 donde se encuentra el archivo Parquet
    s3 = boto3.client('s3')
    
    
    # Descargar el archivo Parquet desde S3
    
    df = wr.s3.read_parquet("s3://henry-lab-142024/dev/", dataset=True)
    
    d_data = df[df['developer'] == developer_name]

    neg_r = int(d_data['malo'].sum())
    pos_r =int( d_data['positivo'].sum())
    
    format = [{developer_name:{'Negativo': neg_r, 'Positivo': pos_r}}]
    
    result = json.dumps(format)
    
    return result

## <div style="color: lightgreen; text-align: center;"> .  def user_data  . </div>

import awswrangler as wr
import boto3
import pandas as pd

def lambda_handler(event, context):
    
    # Obtener el nombre de usuario del evento
    user_v = event['user_1']

    # Conectar al bucket de S3 donde se encuentra el archivo Parquet
    s3 = boto3.client('s3')
    
    # Leer el archivo Parquet desde S3 y crear el DataFrame
    df = wr.s3.read_parquet("s3://henry-lab-142024/user/", dataset=True)

    # Filtrar el DataFrame por el usuario dado
    df_user = df[df['user_id'] == user_v]
    
    # Calcular el dinero gastado y la cantidad de ítems para el usuario dado
    money_spent = df_user['price'].sum()
    money_spent_rounded = str(round(float(money_spent), 2))
    item_count = df_user['item_id'].count()

    # Crear el diccionario con la información
    result = {"Usuario": user_v, "Dinero_gastado": money_spent_rounded, "Cantidad_de_items": int(item_count)}
    
    result_j = pd.DataFrame([result]).T
    # Convertir el diccionario a formato JSON
    final_result = result_j.to_json()

    return final_result

## <div style="color: lightgreen; text-align: center;"> .  def UserForGenre  . </div>

import awswrangler as wr
import boto3
import pandas as pd

def lambda_handler(event, context):
    
    # Obtener el nombre de usuario del evento
    user_v = event['user_1']

    # Conectar al bucket de S3 donde se encuentra el archivo Parquet
    s3 = boto3.client('s3')
    
    # Leer el archivo Parquet desde S3 y crear el DataFrame
    df = wr.s3.read_parquet("s3://henry-lab-142024/user/", dataset=True)

    # Filtrar el DataFrame por el usuario dado
    df_user = df[df['user_id'] == user_v]
    
    # Calcular el dinero gastado y la cantidad de ítems para el usuario dado
    money_spent = df_user['price'].sum()
    money_spent_rounded = str(round(float(money_spent), 2))
    item_count = df_user['item_id'].count()

    # Crear el diccionario con la información
    result = {"Usuario": user_v, "Dinero_gastado": money_spent_rounded, "Cantidad_de_items": int(item_count)}
    
    result_j = pd.DataFrame([result]).T
    # Convertir el diccionario a formato JSON
    final_result = result_j.to_json()

    return final_result

## <div style="color: lightgreen; text-align: center;"> .  def recomendacion_juego  . </div>

In [47]:
#importamos la biblioteca
from sklearn.metrics.pairwise import cosine_similarity

In [85]:
import numpy as np

In [48]:
# Eliminamos la columnas con las que no vamos a trabajar del df
df_processed = df.drop(columns=['developer', 'release_date', 'price']) 

In [49]:
df_processed.set_index('item_id', inplace=True)

In [87]:
df_processed.shape

(49057, 3)

In [50]:
# Cálculo de similitud
similarity_matrix = cosine_similarity(df_processed)

In [ ]:
if not producto.empty:
    product_index = producto.index[0]
    product_similarities = similarity_matrix[product_index]
    most_similar_products_indices = np.argsort(-product_similarities)
    most_similar_products = data.loc[most_similar_products_indices, 'nombre']
    print("Los productos más similares al producto", nombre_del_producto, "son:")
    print(most_similar_products)
else:
    print("Producto no encontrado")

In [88]:
item_id = 10

In [86]:
if not item_id.empty:
    
    product_similarities = similarity_matrix[df[df['item_id'] == item_id]]
    most_similar_products_indices = np.argsort(-product_similarities)
    most_similar_products = data.loc[most_similar_products_indices, 'nombre']
    print("Los productos más similares al producto", nombre_del_producto, "son:")
    print(most_similar_products)
else:
    print("Producto no encontrado")

MemoryError: Unable to allocate 17.9 GiB for an array with shape (1, 49057, 49057) and data type float64

In [99]:
def get_top_similar_items(item_id, n=5):
    item_index = df_processed.index.get_loc(item_id)
    similar_items = similarity_matrix[item_index]
    # Obtener los índices de los elementos más similares en orden descendente
    similar_items_indices = similar_items.argsort()[::-1][:n]
    # Convertir a una lista plana
    similar_items_indices = similar_items_indices.flatten()
    # Excluir el propio elemento y devolver los elementos similares del DataFrame
    return df_processed.iloc[similar_items_indices]

In [106]:
#Hago una prueba
recommended_items = get_top_similar_items(338000)

In [107]:
print(recommended_items.head(5))

         malo  neutral  positivo
item_id                         
200210      0        1         0
440         1        0         0
338000      1        0         0
274190      1        0         0
369580      0        1         0
